## Extract eyes open and eyes closed and stop timestamps to crop data for resting state functional connectivity analysis
#### Input: *stc.mat
####  Output: *.mat or pickle

In [1]:
import sys
import os
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
import mne
import eeg_toolkit
from eeg_toolkit import utils, preprocess
# functional_connectivity
from glob import glob

In [2]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in processed raw data
data_path = '../../../Data/Processed Data/'
epo_path = '../../../Data/Processed Data/' + time_win_path

save_path_EC = '../../Data/FC_EC_resting/'
save_path_EO = '../../Data/FC_EO_resting/'
save_path_zepo = '../../Data/FC_Z_scored_Epochs/' + time_win_path
save_paths = [save_path_EC,save_path_EO,save_path_zepo]

[os.makedirs(path) for path in save_paths if not os.path.exists(path)]

# Noise cov: crop raw during eyes-open resting condition
noise_cov_win = 5.5, 7.5 # rest_min, rest_max

[-2.5,0.0,2.5]


In [8]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_path,os.pardir),'sub_ids.txt')
sub_ids = sub_ids[7:] # select only 64ch subs and turn into set
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [ ]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

In [ ]:
# Load the MATLAB file
sub_id='015'
data_path = f'/../../Data/STC_Z_scored_Epochs/5_sec_time_window/{sub_id}/'

file_path = glob(data_path, '*rh_stc_zepo.mat')
# 'caudalanteriorcingulate-rh_stc_zepo.mat'
data = scipy.io.loadmat(file_path)

# Display the keys and structure of the data
data.keys(), {key: type(data[key]) for key in data.keys()}

# Examine the shape and basic information of the 'data' array
data_shape = data['data'].shape
data_info = data['data']

# Display the shape and some basic info about the data
data_shape, data_info.dtype, data_info.ndim

In [ ]:
# Reshape the data to be compatible with MNE-Connectivity
# Assuming each sample is a separate time point and each trial is independent

# Current shape is (trials, samples), need (trials, channels, samples)
# Since we have only one channel (region), we add an extra dimension
data_reshaped = data['data'][:, np.newaxis, :]

# Create artificial data for a second region (placeholder)
# In real analysis, this should be different data from another region
data_region2 = np.copy(data_reshaped)

# Verify the new shape
data_reshaped.shape, data_region2.shape


In [ ]:
# Assuming data_reshaped is your EEG data reshaped to (trials, channels, samples)
# data_region2 is the placeholder for the second region, identical to data_reshaped

# Concatenate the two regions along the channel dimension
combined_data = np.concatenate([data_reshaped, data_region2], axis=1)

# Create an MNE EpochsArray for compatibility with MNE-Connectivity
info = mne.create_info(['region1', 'region2'], sfreq=100, ch_types='eeg')  # sfreq is the sampling frequency
epochs = mne.EpochsArray(combined_data, info)

# Compute wPLI
fmin, fmax = 8, 13  # Define frequency band (e.g., alpha band)
sfreq = epochs.info['sfreq']  # Sampling frequency
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    epochs, method='wpli2_debiased', mode='multitaper', sfreq=sfreq,
    fmin=fmin, fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

# con contains the wPLI values between the two regions
print(con)
